In [18]:
import pandas as pd
import numpy as np
from json import loads
import pandas as pd
import psycopg2 
import csv
from urllib import parse, request
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
#Extract the features from DB
conn = psycopg2.connect("dbname='SQLBook' user='postgres' host='localhost' password='postgres' password='1234'")
cur = conn.cursor()

#categories: dataframe that contains categories information from Asterixdb (nodeId and 5 levels of categories)
#categories.to_csv('categories.csv')

#reviewFeatures: dataframe that contains review features from Postgres
#reviewFeatures.to_csv('reviewFeatures.csv')

In [3]:
#Function to get review length
def rev_len(x):
    #print (x)
    x['reviewlen']=len(str(x['txt']))
    return x

#Function to get review length
def rev_len(x):
    #print (x)
    x['reviewlen']=len(str(x['txt']))
    return x

#Get number of sentences
def count_sentences(x):
    cnt=0
    num_exc=0
    for i in str(x):
        if i in ['.','?','!']:
            cnt=cnt+1
    for i in str(x):
        if i in ['?','!']:
            num_exc=num_exc+1
    return cnt,num_exc 

#Get average word length
def avg_word_len(x):
    word_list = str(x).split(' ')
    nwords = len(word_list)
    tot_len=0
    for w in word_list:
        tot_len=tot_len+len(w)
    return tot_len/nwords

In [4]:
cur.execute("SELECT r.reviewID,r.asin, cast(r.overall as int) as rating, cast(trim(leading '[' from substring(helpful from 0 for position(',' in helpful))) as int) as votesForReview,cast(trim(trailing ']' from substring(helpful from position(',' in helpful)+2)) as int) as outOf,reviewCount.cnt as reviewerCount,bookCount.cnt as bookreviewCount,EXTRACT(day from current_date - r.reviewTime) age,pr.nodeID,reviewtext,summary FROM reviews r LEFT OUTER JOIN (SELECT reviewID, count(*) cnt FROM reviews GROUP BY reviewID) reviewCount ON r.reviewID = reviewCount.reviewID LEFT OUTER JOIN (SELECT asin, count(*) cnt FROM reviews GROUP BY asin) bookCount ON r.asin = bookCount.asin LEFT OUTER JOIN Products pr ON r.asin = pr.asin")
rows = cur.fetchall()

df = pd.DataFrame(rows,columns=['reviewID','asin','rating','helpfulness','outOf','reviewerCount','bokReviewCount','reviewAge','nodeID','reviewtext','summary'])

#Combining summary and review text
df['txt']=df['reviewtext']+df['summary']
df.drop(['reviewtext','summary'],axis=1,inplace=True)

#Get review length
df['reviewlen']=0
df=df.apply(lambda x: rev_len(x),axis=1)

#Get # of words, word length, # of sentences, ARI index
df['numwords']=0
df['avgwordlen']=0
df['num_sen']=0
df['num_exc']=0
df['ARI']=0

def numwords(x):
    x['numwords']=len(str(x['txt']).split(' '))
    x['avgwordlen'] = avg_word_len(x['txt'])
    x['num_sen'],x['num_exc'] = count_sentences(x['txt'])
    x['ARI'] = 4.71*(len(str(x['txt']))/float(x['numwords']+1)) + 0.5*(x['numwords']/float(x['num_sen']+1)) - 21.43
    return x
df=df.apply(lambda x: numwords(x),axis=1)

#Extract just the feature set from original df
new_df = df[['rating','helpfulness','outOf','reviewerCount','bokReviewCount','reviewAge','reviewlen','numwords','avgwordlen','num_sen','num_exc','ARI']]

In [5]:
#Training data is the reviews with more than 1 outof
train_df=new_df[new_df['outOf']!=0]

#Prediction data is the reviews which dont have any outof
test_df=new_df[new_df['outOf']==0]

#Get the training labels
y_train = train_df['helpfulness']/train_df['outOf']

#Drop the label data from train
train_df.drop(['helpfulness','outOf'],axis=1,inplace=True)

#Split the data into train, validate, test
X_train, X_test, y_train, y_test = train_test_split(train_df, y_train, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=23)

/Users/Alison/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#Normalize the features
scaler = StandardScaler()
scaler.fit(X_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)
X_val_n = scaler.transform(X_val)

In [7]:
# Fit linear regression
reg = LinearRegression()
reg.fit(X_train_n,y_train.values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
# Predict the output for train and validate
y_train_pred = reg.predict(X_train_n)
y_val_pred=reg.predict(X_val_n)

In [9]:
#Check the error for train and validate
mean_squared_error(y_train.values,y_train_pred)
mean_squared_error(y_val.values,y_val_pred)

0.13904183387410762

In [10]:
#Predict for test data
y_test_pred=reg.predict(X_test_n)

#Check the error
mean_squared_error(y_test.values,y_test_pred)

0.13847058382099395

In [11]:
#Actual prediction on unknown labels
test_df.drop(['helpfulness','outOf'],axis=1,inplace=True)

test_df_n = scaler.transform(test_df) 

y_test_df=reg.predict(test_df_n)

#Store the redicted help ratio
test_df['help_ratio']=y_test_df

/Users/Alison/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Alison/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
#Create help ratio in original df
new_df['help_ratio']=new_df['helpfulness']/new_df['outOf']

# Join 
new_df=new_df.join(test_df['help_ratio'],how='left',rsuffix='1')

#Get the final ratio
def final_ratio(x):
    if np.isnan(x['help_ratio']):
        x['help_ratio']=x['help_ratio1']
    return x

new_df=new_df.apply(lambda x: final_ratio(x),axis=1)
new_df.drop('help_ratio1',inplace=True,axis=1)

/Users/Alison/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
df=df.join(new_df['help_ratio'],how='left')
df['rating*help_ratio']=df['rating']*df['help_ratio']
df_grouped = df.groupby('asin').sum()


def get_count(x):
    x['count']=len(df[df['asin']==x['asin']])
    return x

df_grouped['count']=0
df_grouped=df_grouped.reset_index().apply(lambda x: get_count(x),axis=1)

df_grouped['overall_rating1'] = (df_grouped['rating*help_ratio']/df_grouped['count']) + np.log(df_grouped['count'])
#df_grouped=df_grouped.sort_values(by='overall_rating1',ascending=False)

In [14]:
#Get categories mapping, index to category name
cats_map=[]
with open('numcats.txt','r') as f:
    reader=csv.reader(f,delimiter='\t')
    for num,cat in reader:
        cats_map.append(cat)
        
#Get books mapping, index to asin
asin = np.load('asin.npy')

# Asterixdb

In [15]:
class QueryResponse:
    def __init__(self, raw_response):
        self._json = loads(raw_response)

        self.requestID = self._json['requestID'] if 'requestID' in self._json else None
        self.clientContextID = self._json['clientContextID'] if 'clientContextID' in self._json else None
        self.signature = self._json['signature'] if 'signature' in self._json else None
        self.results = self._json['results'] if 'results' in self. _json else None
        self.metrics = self._json['metrics'] if 'metrics' in self._json else None

class AsterixConnection:
    def __init__(self, server = 'http://localhost', port = 19002):
        self._server = server
        self._port = port
        self._url_base = self._server +':'+ str(port)

    def query(self, statement, pretty=False, client_context_id=None):
        endpoint = '/query/service'

        url = self._url_base + endpoint

        payload = {
            'statement': statement,
            'pretty': pretty
        }

        if client_context_id:
            payload['client_context_id'] = client_context_id

        data = parse.urlencode(payload).encode("utf-8")
        req = request.Request(url, data)
        response = request.urlopen(req).read()

        return QueryResponse(response)

In [19]:
#Get category information, grouped by nodeID
asterix_conn = AsterixConnection()
response = asterix_conn.query("""USE TinySocial;
    SELECT 
        nodeID, 
        (SELECT gp.cat.category.nested.level_1, gp.cat.category.nested.nested.level_2, gp.cat.category.nested.nested.nested.level_3, gp.cat.category.nested.nested.nested.nested.level_4, gp.cat.category.nested.nested.nested.nested.nested.level_5 FROM gp) AS categories
    FROM ClassificationInfo c
    GROUP BY c.nodeID AS nodeID
    GROUP AS gp(c as cat)""")
# """

#Parse result from asterixdb and create a dictionary {nodeID: [[lvl1,lvl2,lvl3,lvl4,lvl5],[lvl1,lvl2,..],..]}
max_cat = 0
categories = []

for i in range(len(response.results)):
    if len(response.results[i]['categories']) > max_cat:
        max_cat = len(response.results[i]['categories'])
    c = []
    for x in range(len(response.results[i]['categories'])):
        c.append([cats_map.index(response.results[i]['categories'][x][l]) for l in response.results[i]['categories'][x]])
    categories.append([response.results[i]['nodeID'],c])
    
categories = {i[0]:i[1] + [[0,0,0,0,0]]*(max_cat-len(i[1])) for i in categories}

# Categories Matrix

In [21]:
#Get nodeid to asin mapping
cur.execute("""select nodeid,asin from products""")
rows = cur.fetchall()
nodeid_asin = pd.DataFrame(rows,columns=['nodeid','asin'])
nodeid_asin = nodeid_asin.set_index('asin')

In [22]:
#Reorder categories with the nodeid to asin mapping (from postgres) and index to asin mapping (from asin.npy)
categories_indexed = []

for i in range(max_cat):
    c = []
    for a in asin:
        c.append(categories[int(nodeid_asin.ix[a.decode()]['nodeid'])][i])
    categories_indexed.append(c)

# Get seasonal information

In [23]:
cur.execute("""select asin,
          round(100*sum(case when month >= 3 and month < 6 then numunits else 0 end)/sum(numunits),2) as spring,
          round(100*sum(case when month >= 6 and month < 9 then numunits else 0 end)/sum(numunits),2) as summer,
          round(100*sum(case when month >= 9 and month < 12 then numunits else 0 end)/sum(numunits),2) as fall,
          round(100*sum(case when (month = 12 or month < 3) then numunits else 0 end)/sum(numunits),2) as winter,
          cast(max(fullprice) as decimal) fullprice,
          case when max(isinstock) = 'Y' then 1 else 0 end isinstock
          from 
          (select asin, EXTRACT(MONTH FROM orderdate) as month, case when l.numunits = 0 then 0.00001 else l.numunits end as numunits, fullprice, isinstock
          from customers c, orders o, orderlines l, products p
          where c.customerid = o.customerid
          and o.orderid = l.orderid
          and l.productid = p.productid
          ) as temp
          group by asin""")
# """
rows = cur.fetchall()

In [24]:
season = pd.DataFrame(rows,columns=['asin','spring','summer','fall','winter','fullprice','isinstock'])

In [45]:
asin_final = []

for i in asin:
    if i.decode() in df_grouped.asin.values:
        row = [i for i in df_grouped[df_grouped.asin==i.decode()][['overall_rating1','count']].values[0]]
    else:
        row = [0,0]
        
    if i.decode() in season.asin.values:
        row += [i for i in season[season.asin==i.decode()][['spring','summer','fall','winter','fullprice','isinstock']].values[0]]
    else:
        row += [0,0,0,0,0,'']
        
    asin_final.append(row)

In [46]:
#overall_rating, count, spring, summer, fall, winter, price, instock
asin_final[0:5]

[[3.1680567331125546,
  3.0,
  Decimal('0.00'),
  Decimal('33.33'),
  Decimal('0.00'),
  Decimal('66.67'),
  Decimal('540.00'),
  1],
 [2.0769230769230766,
  1.0,
  Decimal('0.00'),
  Decimal('33.33'),
  Decimal('33.33'),
  Decimal('33.33'),
  Decimal('306.00'),
  1],
 [4.8026425700974524,
  3.0,
  Decimal('25.00'),
  Decimal('0.00'),
  Decimal('25.00'),
  Decimal('50.00'),
  Decimal('340.00'),
  1],
 [5.6732632446133442,
  28.0,
  Decimal('0.00'),
  Decimal('100.00'),
  Decimal('0.00'),
  Decimal('0.00'),
  Decimal('375.00'),
  1],
 [2.2219158793519567,
  2.0,
  Decimal('0.00'),
  Decimal('100.00'),
  Decimal('0.00'),
  Decimal('0.00'),
  Decimal('540.00'),
  1]]

In [75]:
rating_indexed = np.array(asin_final)[:,0]

In [94]:
rating_indexed[rating_indexed==0] = np.min(rating_indexed[rating_indexed != 0])/100

In [96]:
np.save("rating_indexed.npy",rating_indexed)

In [69]:
np.save('season_price_instock_indexed.npy',np.array(asin_final, dtype=np.float)[:,2:])